In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates
from ipywidgets import interact, Dropdown, VBox, Output


# Charger vos données
file_path = 'PN1502_02_DET_PLG_034123.xlsx'
df = pd.read_excel(file_path)

# S'assurer que les dates sont au format datetime
df['Date arbitrée'] = pd.to_datetime(df['Date arbitrée MOA'], errors='coerce')
df['EDL 1'] = pd.to_datetime(df['Visite 1'], errors='coerce')
df['EDL 2'] = pd.to_datetime(df['Visite 2'], errors='coerce')

# Liste des sites pour le menu déroulant
sites = df['Site'].unique()

# Créer un widget Output pour afficher le graphique
output = Output()

# Fonction pour mettre à jour le graphique en fonction du site sélectionné
def update_graph(site):
    with output:
        # Effacer le graphique précédent
        output.clear_output()
        
        # Filtrer les données pour le site spécifique
        df_filtered_ouvrage = df[df['Site'] == site]

        # Créer une plage de dates allant du minimum au maximum des dates pertinentes
        toutes_les_dates = pd.date_range(start=df_filtered_ouvrage[['Date arbitrée', 'EDL 1', 'EDL 2']].min().min(),
                                         end=df_filtered_ouvrage[['Date arbitrée', 'EDL 1', 'EDL 2']].max().max(),
                                         freq='D')

        # Créer un DataFrame avec toutes les dates
        df_dates = pd.DataFrame({'dates': toutes_les_dates})

        # Fusionner les données pour 'Date arbitrée'
        df_arbitree = df_dates.merge(df_filtered_ouvrage[['Date arbitrée', 'Local(s) Technique']], left_on='dates', right_on='Date arbitrée', how='left')
        df_arbitree = df_arbitree.rename(columns={'Date arbitrée': 'Date arbitrée'})

        # Fusionner les données pour 'EDL 1'
        df_edl1 = df_dates.merge(df_filtered_ouvrage[['EDL 1', 'Local(s) Technique']], left_on='dates', right_on='EDL 1', how='left')
        df_edl1 = df_edl1.rename(columns={'EDL 1': 'EDL 1'})

        # Fusionner les données pour 'EDL 2'
        df_edl2 = df_dates.merge(df_filtered_ouvrage[['EDL 2', 'Local(s) Technique']], left_on='dates', right_on='EDL 2', how='left')
        df_edl2 = df_edl2.rename(columns={'EDL 2': 'EDL 2'})

        # Concaténer les trois DataFrames
        df_final = pd.concat([df_arbitree, df_edl1, df_edl2], axis=0, ignore_index=True)

        # Supprimer les lignes dupliquées
        df_final = df_final.drop_duplicates(subset=['dates', 'Local(s) Technique', 'EDL 1', 'EDL 2'])

        # Trier les données par dates
        df_final_sorted = df_final.sort_values(by='dates').reset_index(drop=True)

        # Filtrer les données pour exclure les lignes spécifiques
        axe_x_arbitrees = df_final_sorted['dates']
        df_final_sorted['axe_y_arbitrees'] = np.where(df_final_sorted['Date arbitrée'].notna(), df_final_sorted['Local(s) Technique'], np.nan)
        axe_y_arbitrees = df_final_sorted['axe_y_arbitrees']
        axe_y_arbitrees.fillna("poubelle", inplace=True)

        axe_x_edl_1 = df_final_sorted['dates']
        df_final_sorted['axe_y_edl_1'] = np.where(df_final_sorted['EDL 1'].notna(), df_final_sorted['Local(s) Technique'], np.nan)
        axe_y_edl_1 = df_final_sorted['axe_y_edl_1']
        axe_y_edl_1.fillna("poubelle", inplace=True)

        axe_x_edl_2 = df_final_sorted['dates']
        df_final_sorted['axe_y_edl_2'] = np.where(df_final_sorted['EDL 2'].notna(), df_final_sorted['Local(s) Technique'], np.nan)
        axe_y_edl_2 = df_final_sorted['axe_y_edl_2']
        axe_y_edl_2.fillna("poubelle", inplace=True)

        # Filtrer les données pour exclure les lignes spécifiques
        mask = axe_y_arbitrees != 'poubelle'
        axe_x_arbitrees_filtered = axe_x_arbitrees[mask]
        axe_y_arbitrees_filtered = axe_y_arbitrees[mask]

        mask = axe_y_edl_1 != 'poubelle'
        axe_x_edl_1_filtered = axe_x_edl_1[mask]
        axe_y_edl_1_filtered = axe_y_edl_1[mask]

        mask = axe_y_edl_2 != 'poubelle'
        axe_x_edl_2_filtered = axe_x_edl_2[mask]
        axe_y_edl_2_filtered = axe_y_edl_2[mask]

        # Création de la figure et de l'axe principal
        fig, ax = plt.subplots(figsize=(11, 6))
        plt.subplots_adjust(bottom=0.25)

        # Tracé du graphique
        sc_arbitre = ax.scatter(axe_x_arbitrees_filtered, axe_y_arbitrees_filtered, color='magenta', marker='o', label='Date arbitrée initialement')
        sc_edl_1 = ax.scatter(axe_x_edl_1_filtered, axe_y_edl_1_filtered, color='b', marker='x', label='Date EDL 1')
        sc_edl_2 = ax.scatter(axe_x_edl_2_filtered, axe_y_edl_2_filtered, color='black', marker='x', label='Date EDL 2')
        ax.set_xlabel('Dates')
        ax.set_ylabel('LT')
        ax.set_title('Timeline MàD')
        ax.grid(True)
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        plt.xticks(rotation=45)
        plt.tight_layout()

        # Ajouter la légende
        ax.legend()

        # Ajouter la légende à l'extérieur du graphique
        ax.legend(bbox_to_anchor=(1, -0.2), loc='lower right')

        # Définir les étiquettes des ticks de l'axe Y
        #LT_unique = axe_y_arbitrees_filtered.unique()
        #nb_LT = len(LT_unique)
        #cst = 2
        #yticks = list(range(0-cst, nb_LT-cst, 1))[::-1]

        # Afficher les étiquettes personnalisées à côté des ticks
        #for i, label in zip(yticks, LT_unique):
            #ax.text(-0.3, i, label, ha='right', va='baseline', transform=ax.get_yaxis_transform(), fontsize=12)

        # Ajout d'annotations interactives avec mpld3
        #labels = [f'{date:%Y-%m-%d} ; {value}' for date, value in zip(axe_x_arbitrees_filtered, axe_y_arbitrees_filtered)]
        #tooltip = mpld3.plugins.PointLabelTooltip(sc_arbitre, labels=labels)
        #mpld3.plugins.connect(fig, tooltip)

        #labels = [f'{date:%Y-%m-%d} ; {value}' for date, value in zip(axe_x_edl_1_filtered, axe_y_edl_1_filtered)]
        #tooltip = mpld3.plugins.PointLabelTooltip(sc_edl_1, labels=labels)
        #mpld3.plugins.connect(fig, tooltip)

        #labels = [f'{date:%Y-%m-%d} ; {value}' for date, value in zip(axe_x_edl_2_filtered, axe_y_edl_2_filtered)]
        #tooltip = mpld3.plugins.PointLabelTooltip(sc_edl_2, labels=labels)
        #mpld3.plugins.connect(fig, tooltip)

        plt.show()

# Création du menu déroulant pour sélectionner le site
dropdown = Dropdown(options=sites, description='Site:', value=sites[0])

# Afficher le menu déroulant et le graphique côte à côte
ui = VBox([dropdown, output])
display(ui)

# Lier la fonction de mise à jour au changement de valeur du menu déroulant
dropdown.observe(lambda change: update_graph(change['new']), names='value')

# Initialiser avec le premier site
update_graph(sites[0])
